In [15]:
from pathlib import Path
import json
import shutil
import pickle
import numpy as np
import pandas as pd
from prophet import Prophet
from pandas.tseries.holiday import USFederalHolidayCalendar
from prophet.serialize import model_from_json

# ================== INPUTS ==================
HOURLY_PATH = Path("cluster_hourly.parquet")

SARIMAX_DIR = Path("sarimax/sarimax_best")
PROPHET_DIR = Path("prophet_folder/prophet_best")

CLUSTERS = [0, 1]

# We evaluate day-ahead forecasts for these months
VAL_MONTH = 10           # October
TEST_MONTHS = [11, 12]   # Nov-Dec

HARD_END = pd.Timestamp("2018-12-31 23:00:00")

# Regressors used in training scripts
LAGS = [24, 168]  # hours
USE_US_HOLIDAYS_SARIMAX = True

# Prophet used weekend/weekday conditional seasonality + lag regressors
USE_LAGS_PROPHET = True

# ================== OUTPUTS ==================
PRED_OUT_DIR = Path("preds_ts_daily")
ART_SARIMAX_DIR = Path("artifacts_sarimax_daily")
ART_PROPHET_DIR = Path("artifacts_prophet_daily")
PRED_OUT_DIR.mkdir(parents=True, exist_ok=True)
ART_SARIMAX_DIR.mkdir(parents=True, exist_ok=True)
ART_PROPHET_DIR.mkdir(parents=True, exist_ok=True)

print("[INFO] Outputs ->", PRED_OUT_DIR.resolve())


[INFO] Outputs -> /mnt/data/Desktop/Masters/Business_Analytics/Intro_to_BA/preds_ts_daily


In [16]:
df = pd.read_parquet(HOURLY_PATH).copy()
print("[INFO] Loaded:", df.shape)
print("[INFO] Columns:", list(df.columns))

# cluster
if "cluster" not in df.columns:
    if "cluster_id" in df.columns:
        df = df.rename(columns={"cluster_id":"cluster"})
    elif "gmm20_cluster" in df.columns:
        df = df.rename(columns={"gmm20_cluster":"cluster"})
    else:
        raise ValueError("Need cluster column: 'cluster' or 'cluster_id' or 'gmm20_cluster'.")

# datetime_hour
if "datetime_hour" not in df.columns:
    if "timestamp" in df.columns:
        df = df.rename(columns={"timestamp":"datetime_hour"})
    elif "datetime" in df.columns:
        df = df.rename(columns={"datetime":"datetime_hour"})
    elif {"date","hour"}.issubset(df.columns):
        df["datetime_hour"] = pd.to_datetime(df["date"]).dt.normalize() + pd.to_timedelta(df["hour"], unit="h")
    else:
        raise ValueError("Need time column: 'datetime_hour' or 'timestamp'/'datetime' or ('date','hour').")

df["datetime_hour"] = pd.to_datetime(df["datetime_hour"], errors="coerce")
df = df.dropna(subset=["datetime_hour"]).copy()

# departures/arrivals
if "departures" not in df.columns:
    if "pickups" in df.columns:
        df = df.rename(columns={"pickups":"departures"})
    else:
        raise ValueError("Need departures column: 'departures' or 'pickups'.")

if "arrivals" not in df.columns:
    if "dropoffs" in df.columns:
        df = df.rename(columns={"dropoffs":"arrivals"})
    else:
        raise ValueError("Need arrivals column: 'arrivals' or 'dropoffs'.")

# filter
df = df[df["cluster"].isin(CLUSTERS)].copy()
df = df[df["datetime_hour"] <= HARD_END].copy()
df = df.sort_values(["cluster","datetime_hour"]).reset_index(drop=True)

print("[INFO] After filtering:", df.shape)
df.head()


[INFO] Loaded: (183960, 10)
[INFO] Columns: ['cluster', 'datetime_hour', 'departures', 'arrivals', 'was_missing_departures', 'was_missing_arrivals', 'weekday', 'hour', 'month', 'year']
[INFO] After filtering: (17520, 10)


,cluster,datetime_hour,departures,arrivals,was_missing_departures,was_missing_arrivals,weekday,hour,month,year
0,0,2018-01-01 00:00:00,3.0,1.0,False,False,0,0,1,2018
1,0,2018-01-01 01:00:00,2.0,4.0,False,False,0,1,1,2018
2,0,2018-01-01 02:00:00,3.0,2.0,False,False,0,2,1,2018
3,0,2018-01-01 03:00:00,4.0,7.0,False,False,0,3,1,2018
4,0,2018-01-01 04:00:00,0.0,1.0,False,False,0,4,1,2018


In [17]:
def make_dense_series(df_c: pd.DataFrame, col_y: str) -> pd.Series:
    tmp = df_c[["datetime_hour", col_y]].copy()
    tmp = tmp.groupby("datetime_hour", as_index=False)[col_y].sum().sort_values("datetime_hour")
    idx = pd.date_range(tmp["datetime_hour"].min(), tmp["datetime_hour"].max(), freq="h")
    out = pd.DataFrame({"ds": idx}).merge(tmp.rename(columns={"datetime_hour":"ds"}), on="ds", how="left")
    out[col_y] = out[col_y].fillna(0.0).astype(float)
    s = out.set_index("ds")[col_y].asfreq("H")
    if s.isna().any():
        raise ValueError("Dense series still has NaNs — unexpected.")
    return s

series = {}
for c in CLUSTERS:
    df_c = df[df["cluster"] == c].copy()
    series[c] = {
        "departures": make_dense_series(df_c, "departures"),
        "arrivals": make_dense_series(df_c, "arrivals"),
    }
    print(f"[INFO] cluster {c}: departures hours={len(series[c]['departures'])}, arrivals hours={len(series[c]['arrivals'])}")


[INFO] cluster 0: departures hours=8760, arrivals hours=8760
[INFO] cluster 1: departures hours=8760, arrivals hours=8760


/tmp/ipykernel_809108/2085566942.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = out.set_index("ds")[col_y].asfreq("H")
/tmp/ipykernel_809108/2085566942.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = out.set_index("ds")[col_y].asfreq("H")
/tmp/ipykernel_809108/2085566942.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = out.set_index("ds")[col_y].asfreq("H")
/tmp/ipykernel_809108/2085566942.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = out.set_index("ds")[col_y].asfreq("H")


In [18]:
def find_one(pattern: str, base: Path) -> Path:
    m = sorted(base.glob(pattern))
    if not m:
        raise FileNotFoundError(f"No match for {pattern} in {base}")
    return m[0]

sarimax_paths = {c: find_one(f"sarimax_departures_model_{c}_*.pkl", SARIMAX_DIR) for c in CLUSTERS}
prophet_paths = {c: find_one(f"prophet_model_{c}_*.json", PROPHET_DIR) for c in CLUSTERS}

print("SARIMAX paths:", {k:v.name for k,v in sarimax_paths.items()})
print("PROPHET paths:", {k:v.name for k,v in prophet_paths.items()})

sarimax_models = {}
for c,p in sarimax_paths.items():
    with open(p, "rb") as f:
        sarimax_models[c] = pickle.load(f)

prophet_models = {}
for c,p in prophet_paths.items():
    prophet_models[c] = model_from_json(p.read_text())

print("[OK] Loaded SARIMAX + Prophet models.")


SARIMAX paths: {0: 'sarimax_departures_model_0_sarimax_dep_p1d1q2_P1D0Q1_S24.pkl', 1: 'sarimax_departures_model_1_sarimax_dep_p1d1q1_P1D0Q1_S24.pkl'}
PROPHET paths: {0: 'prophet_model_0_prophet_additive_cps0.2_sps10_dfo25_wfo9.json', 1: 'prophet_model_1_prophet_additive_cps0.2_sps5_dfo25_wfo10.json'}
[OK] Loaded SARIMAX + Prophet models.


In [19]:
def us_holiday_indicator(idx: pd.DatetimeIndex) -> pd.Series:
    cal = USFederalHolidayCalendar()
    hols = cal.holidays(start=idx.min().floor("D"), end=idx.max().ceil("D"))
    return idx.floor("D").isin(hols).astype(int)

def prophet_conditions(df_future: pd.DataFrame) -> pd.DataFrame:
    dow = df_future["ds"].dt.dayofweek
    df_future["is_weekend"] = (dow >= 5)
    df_future["is_weekday"] = (dow < 5)
    return df_future

def build_lag_regressors_from_history(future_idx: pd.DatetimeIndex, history: pd.Series, lags=LAGS) -> pd.DataFrame:
    """
    For each future timestamp t, create y_lag_L = history[t-L].
    This is leak-free if history only contains values <= cutoff-1h and t is in next-day horizon.
    """
    exog = pd.DataFrame(index=future_idx)
    for L in lags:
        exog[f"y_lag_{L}"] = [float(history.get(t - pd.Timedelta(hours=L), np.nan)) for t in future_idx]
    return exog


In [20]:
def list_cutoffs_for_months(year=2018, months=[10,11,12]) -> list[pd.Timestamp]:
    cutoffs = []
    for m in months:
        start = pd.Timestamp(year=year, month=m, day=1, hour=0)
        if m == 12:
            end = pd.Timestamp(year=2019, month=1, day=1, hour=0)
        else:
            end = pd.Timestamp(year=year, month=m+1, day=1, hour=0)
        # these are cutoffs at midnight for each day
        cutoffs.extend(list(pd.date_range(start, end - pd.Timedelta(days=1), freq="D")))
    return cutoffs

val_cutoffs = list_cutoffs_for_months(2018, [VAL_MONTH])
test_cutoffs = list_cutoffs_for_months(2018, TEST_MONTHS)

print("[INFO] #val days:", len(val_cutoffs), "first:", val_cutoffs[0], "last:", val_cutoffs[-1])
print("[INFO] #test days:", len(test_cutoffs), "first:", test_cutoffs[0], "last:", test_cutoffs[-1])


[INFO] #val days: 31 first: 2018-10-01 00:00:00 last: 2018-10-31 00:00:00
[INFO] #test days: 61 first: 2018-11-01 00:00:00 last: 2018-12-31 00:00:00


In [21]:
def sarimax_daily_forecast(res, history_y: pd.Series, cutoff: pd.Timestamp) -> pd.Series:
    """
    Forecast next 24 hours starting at cutoff using SARIMAX results.
    Exog: y_lag_24/y_lag_168 (from history_y) + is_us_holiday.
    """
    horizon = pd.date_range(cutoff, cutoff + pd.Timedelta(hours=23), freq="h")
    exog = build_lag_regressors_from_history(horizon, history_y, lags=LAGS)

    if USE_US_HOLIDAYS_SARIMAX:
        exog["is_us_holiday"] = us_holiday_indicator(horizon)


    # SARIMAX expects 2D array-like exog aligned with steps
    fc = res.get_forecast(steps=len(horizon), exog=exog)
    pred = fc.predicted_mean
    pred.index = horizon
    return pred


In [22]:
def prophet_daily_forecast(model, history_y: pd.Series, cutoff: pd.Timestamp) -> pd.Series:
    horizon = pd.date_range(cutoff, cutoff + pd.Timedelta(hours=23), freq="h")
    future = pd.DataFrame({"ds": horizon})
    future = prophet_conditions(future)

    if USE_LAGS_PROPHET:
        exog = build_lag_regressors_from_history(horizon, history_y, lags=LAGS)
        for col in exog.columns:
            future[col] = exog[col].values

        # If any lag is missing, cannot forecast that day safely
        if future[[f"y_lag_{L}" for L in LAGS]].isna().any().any():
            # return NaNs to be handled upstream
            return pd.Series(index=horizon, dtype=float)

    fcst = model.predict(future)
    yhat = pd.Series(fcst["yhat"].values, index=horizon)
    return yhat


In [23]:
rows = []

def add_one_split(cluster_id: int, split_name: str, cutoffs: list[pd.Timestamp]):
    y_dep = series[cluster_id]["departures"]
    y_arr = series[cluster_id]["arrivals"]

    sar_res = sarimax_models[cluster_id]
    pr_model = prophet_models[cluster_id]

    for cutoff in cutoffs:
        # history available up to cutoff-1h
        hist_end = cutoff - pd.Timedelta(hours=1)
        y_dep_hist = y_dep.loc[:hist_end]
        y_arr_hist = y_arr.loc[:hist_end]

        # horizon = next 24h of THIS day (starting at cutoff)
        horizon = pd.date_range(cutoff, cutoff + pd.Timedelta(hours=23), freq="h")

        # ground truth (allowed for evaluation)
        y_true_pickups = y_dep.loc[horizon].values
        y_true_dropoffs = y_arr.loc[horizon].values

        # predictions (leak-free)
        y_pred_pick = sarimax_daily_forecast(sar_res, y_dep_hist, cutoff).values
        y_pred_drop = prophet_daily_forecast(pr_model, y_arr_hist, cutoff).values

        for ts, hr, yt_p, yp_p, yt_d, yp_d in zip(horizon, horizon.hour, y_true_pickups, y_pred_pick, y_true_dropoffs, y_pred_drop):
            rows.append({
                "date": ts.normalize(),
                "hour": int(hr),
                "cluster_id": int(cluster_id),
                "split": split_name,
                "y_true_pickups": float(yt_p),
                "y_pred_sarimax_pickups": float(yp_p) if np.isfinite(yp_p) else np.nan,
                "y_true_dropoffs": float(yt_d),
                "y_pred_prophet_dropoffs": float(yp_d) if np.isfinite(yp_d) else np.nan,
            })

for c in CLUSTERS:
    add_one_split(c, "val", val_cutoffs)
    add_one_split(c, "test", test_cutoffs)

out = pd.DataFrame(rows)

# clip negatives (demand can't be negative)
out["y_pred_sarimax_pickups"] = out["y_pred_sarimax_pickups"].clip(lower=0)
out["y_pred_prophet_dropoffs"] = out["y_pred_prophet_dropoffs"].clip(lower=0)

# errors
out["ae_pickups_sarimax"] = (out["y_true_pickups"] - out["y_pred_sarimax_pickups"]).abs()
out["ae_dropoffs_prophet"] = (out["y_true_dropoffs"] - out["y_pred_prophet_dropoffs"]).abs()

print("[INFO] out shape:", out.shape)
out.head()


[INFO] out shape: (4416, 10)


,date,hour,cluster_id,split,y_true_pickups,y_pred_sarimax_pickups,y_true_dropoffs,y_pred_prophet_dropoffs,ae_pickups_sarimax,ae_dropoffs_prophet
0,2018-10-01,0,0,val,15.0,16.343159,15.0,18.783953,1.343159,3.783953
1,2018-10-01,1,0,val,6.0,9.754439,11.0,9.974062,3.754439,1.025938
2,2018-10-01,2,0,val,3.0,2.230385,3.0,5.888411,0.769615,2.888411
3,2018-10-01,3,0,val,1.0,0.840597,0.0,5.387385,0.159403,5.387385
4,2018-10-01,4,0,val,2.0,0.000000,3.0,4.569315,2.000000,1.569315


In [24]:
for c in CLUSTERS:
    dfc = out[out["cluster_id"] == c].copy().sort_values(["date","hour"]).reset_index(drop=True)

    cols = [
        "date","hour","cluster_id","split",
        "y_true_pickups","y_pred_sarimax_pickups","ae_pickups_sarimax",
        "y_true_dropoffs","y_pred_prophet_dropoffs","ae_dropoffs_prophet",
    ]
    dfc = dfc[cols]

    csv_path = PRED_OUT_DIR / f"ts_daily_cluster_{c}_preds.csv"
    pq_path  = PRED_OUT_DIR / f"ts_daily_cluster_{c}_preds.parquet"

    dfc.to_csv(csv_path, index=False)
    dfc.to_parquet(pq_path, index=False)

    print(f"[OK] Saved cluster {c}:")
    print("  ", csv_path)
    print("  ", pq_path)


[OK] Saved cluster 0:
   preds_ts_daily/ts_daily_cluster_0_preds.csv
   preds_ts_daily/ts_daily_cluster_0_preds.parquet
[OK] Saved cluster 1:
   preds_ts_daily/ts_daily_cluster_1_preds.csv
   preds_ts_daily/ts_daily_cluster_1_preds.parquet


In [25]:
for c in CLUSTERS:
    shutil.copy2(sarimax_paths[c], ART_SARIMAX_DIR / sarimax_paths[c].name)
    shutil.copy2(prophet_paths[c], ART_PROPHET_DIR / prophet_paths[c].name)

sarimax_meta = {
    "model": "SARIMAX",
    "target": "pickups",
    "clusters": CLUSTERS,
    "lags": LAGS,
    "use_us_holidays": USE_US_HOLIDAYS_SARIMAX,
    "evaluation": "rolling daily 24h (cutoff at midnight)",
    "val_month": VAL_MONTH,
    "test_months": TEST_MONTHS,
    "files": {str(c): sarimax_paths[c].name for c in CLUSTERS},
}

prophet_meta = {
    "model": "Prophet",
    "target": "dropoffs",
    "clusters": CLUSTERS,
    "lags": LAGS,
    "evaluation": "rolling daily 24h (cutoff at midnight)",
    "val_month": VAL_MONTH,
    "test_months": TEST_MONTHS,
    "files": {str(c): prophet_paths[c].name for c in CLUSTERS},
}

(ART_SARIMAX_DIR / "sarimax_meta.json").write_text(json.dumps(sarimax_meta, indent=2))
(ART_PROPHET_DIR / "prophet_meta.json").write_text(json.dumps(prophet_meta, indent=2))

print("[OK] Artifacts + metadata written.")


[OK] Artifacts + metadata written.


In [26]:
def summarize(split):
    d = out[out["split"] == split]
    return {
        "rows": len(d),
        "mae_pickups_sarimax": float(d["ae_pickups_sarimax"].mean()),
        "mae_dropoffs_prophet": float(d["ae_dropoffs_prophet"].mean()),
    }

print("VAL:", summarize("val"))
print("TEST:", summarize("test"))


VAL: {'rows': 1488, 'mae_pickups_sarimax': 17.78247152562429, 'mae_dropoffs_prophet': 14.345435894657854}
TEST: {'rows': 2928, 'mae_pickups_sarimax': 16.392839568616655, 'mae_dropoffs_prophet': 16.798808070995065}
